In [1]:
"""
Relevance Detection
"""

'\nRelevance Detection\n'

In [2]:
import pandas as pd
import nltk
import numpy as np
import preprocessing
import importlib
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from collections import Counter
from sklearn.metrics import confusion_matrix
import score
from sklearn.preprocessing import StandardScaler

In [25]:
#run this cell to reload the preprocessing module
importlib.reload(preprocessing)

<module 'preprocessing' from '/Users/dannyyang/Documents/GitHub/Insights-FakeNews/preprocessing.py'>

In [4]:
train_stances = pd.read_csv("fn_data/train_stances.csv")
print(train_stances.shape)
train_stances.head()

(49972, 3)


,Headline,Body ID,Stance
0,Police find mass graves with at least '15 bodi...,712,unrelated
1,Hundreds of Palestinians flee floods in Gaza a...,158,agree
2,"Christian Bale passes on role of Steve Jobs, a...",137,unrelated
3,HBO and Apple in Talks for $15/Month Apple TV ...,1034,unrelated
4,Spider burrowed through tourist's stomach and ...,1923,disagree


In [5]:
train_bodies = pd.read_csv("fn_data/train_bodies.csv")
print(train_bodies.shape)
train_bodies.head()

(1683, 2)


,Body ID,articleBody
0,0,A small meteorite crashed into a wooded area i...
1,4,Last week we hinted at what was to come as Ebo...
2,5,(NEWSER) – Wonder how long a Quarter Pounder w...
3,6,"Posting photos of a gun-toting child online, I..."
4,7,At least 25 suspected Boko Haram insurgents we...


In [6]:
stances_tr, stances_val = preprocessing.train_test_split(train_bodies, train_stances)
stances_tr.shape, stances_val.shape

((39949, 3), (10023, 3))

In [7]:
# this one takes a while!
idf = preprocessing.build_idf(train_bodies, stances_tr)

4.8233617373283293

In [72]:
#this is just a comparison between using IDF score and not using IDF score - not related to the model
#change the body id to see
body = preprocessing.get_body(5, train_bodies)
#no IDF
processed2 = preprocessing.process_body(body)
print(processed2['common_nouns'],processed2['common_verbs'])

#with IDF
processed = preprocessing.process_body(body, idf)
print(processed['common_nouns'],processed['common_verbs'])

['burger', 'year', 'friend', 'news', 'report'] ['bought', 'showed', 'started', 'wonder', 'went']
['burger', 'year', 'friend', 'charity', 'mcjordan'] ['bought', 'started', 'showed', 'dissuaded', 'sauce']


In [80]:
body = preprocessing.get_body(1369, train_bodies)
processed = preprocessing.process_body(body, idf)
print(processed['first_sentence']['tokens'])
print(processed['significant_sentence']['tokens'])
print(processed['first_sentence']['adverbs'],processed['significant_sentence']['adverbs'])
print(processed['first_sentence']['adjectives'],processed['significant_sentence']['adjectives'])
print(processed['first_sentence']['verbs'],processed['significant_sentence']['verbs'])

['unconfirmed', 'report', 'circulating', 'social', 'medium', 'islamic', 'state', 'group', 'carried', 'chemical', 'attack', 'battling', 'kurdish', 'force', 'kobani']
['kurdish', 'affair', 'analyst', 'mutlu', 'civiroglu', 'spoke', 'remaining', 'doctor', 'inside', 'kobani', 'told', 'victim', 'civilian']
[] []
['unconfirmed', 'social', 'islamic', 'chemical', 'kurdish'] ['kurdish', 'civiroglu']
['circulating', 'carried', 'battling'] ['spoke', 'remaining', 'told']


In [20]:
#this takes a while!
body_info = preprocessing.process_bodies(train_bodies, idf)

processed 100
processed 200
processed 300
processed 400
processed 500
processed 600
processed 700
processed 800
processed 900
processed 1000
processed 1100
processed 1200
processed 1300
processed 1400
processed 1500
processed 1600
done! processed 1683


In [21]:
feats_list = [
    'shared_nouns',
    'shared_verbs',
    'shared_bigrams',
    'shared_trigrams',
    'shared_tokens',
#     'sentiment_pos',
#     'sentiment_neg',
#     'sentiment_neu',
#     'sentiment_compound',

    'shared_nouns_fst',
    'shared_verbs_fst',
    'shared_bigrams_fst',
    'shared_trigrams_fst',
    'shared_tokens_fst',
#     'sentiment_pos_fst',
#     'sentiment_neg_fst',
#     'sentiment_neu_fst',
#     'sentiment_compound_fst',

    'shared_nouns_sig',
    'shared_verbs_sig',
    'shared_bigrams_sig',
    'shared_trigrams_sig',
    'shared_tokens_sig',
#     'sentiment_pos_sig',
#     'sentiment_neg_sig',
#     'sentiment_neu_sig',
#     'sentiment_compound_sig',
    
    'cos_nouns_sig',
    'cos_bigrams_sig',
    'cos_trigrams_sig',
    'cos_tokens_sig',

    'cos_nouns_fst',
    'cos_bigrams_fst',
    'cos_trigrams_fst',
    'cos_tokens_fst',
]

In [26]:
# this one takes a while also!

#train data
data_feats = [preprocessing.get_feats(i, body_info) for i in stances_tr.values]
train_df = pd.DataFrame()
for i in feats_list:
    train_df[i] = [x[i] for x in data_feats]

#val data
val_feats = [preprocessing.get_feats(i, body_info) for i in stances_val.values]
val_df = pd.DataFrame()
for i in feats_list:
    val_df[i] = [x[i] for x in val_feats]

In [27]:
train_df.head()

,shared_nouns,shared_verbs,shared_bigrams,shared_trigrams,shared_tokens,shared_nouns_fst,shared_verbs_fst,shared_bigrams_fst,shared_trigrams_fst,shared_tokens_fst,...,shared_trigrams_sig,shared_tokens_sig,cos_nouns_sig,cos_bigrams_sig,cos_trigrams_sig,cos_tokens_sig,cos_nouns_fst,cos_bigrams_fst,cos_trigrams_fst,cos_tokens_fst
0,0,0,0,0,0,0,0,0,0,0,...,0,0,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000
1,0,0,0,0,0,0,0,0,0,0,...,0,0,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000
2,0,0,0,0,0,0,0,0,0,0,...,0,0,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000
3,1,0,0,0,3,0,0,0,0,0,...,0,1,0.666667,1.000000,1.000000,0.841886,1.00000,1.000000,1.000000,1.000000
4,4,0,0,0,12,3,0,2,1,8,...,6,9,0.087129,0.332576,0.367544,0.216651,0.53709,0.838835,0.912294,0.403715


In [28]:
train_df['label'] = [-1 if x == "unrelated" else 1 for x in list(stances_tr['Stance'])]
val_df['label'] = [-1 if x == "unrelated" else 1 for x in list(stances_val['Stance'])]
Counter(train_df['label'])

Counter({-1: 29099, 1: 10850})

In [62]:
scaler = StandardScaler()

for i in feats_list:
    train_df[i] = scaler.fit_transform(train_df[i].values.reshape(-1,1))
    val_df[i] = scaler.fit_transform(val_df[i].values.reshape(-1,1))
train_df.head()

,shared_nouns,shared_verbs,shared_bigrams,shared_trigrams,shared_tokens,shared_nouns_fst,shared_verbs_fst,shared_bigrams_fst,shared_trigrams_fst,shared_tokens_fst,...,shared_tokens_sig,cos_nouns_sig,cos_bigrams_sig,cos_trigrams_sig,cos_tokens_sig,cos_nouns_fst,cos_bigrams_fst,cos_trigrams_fst,cos_tokens_fst,label
0,-0.402182,-0.211633,0.0,0.0,-0.738752,-0.497202,-0.258563,-0.346181,-0.164315,-0.544335,...,-0.429451,0.342679,0.223577,0.083049,0.445390,0.442750,0.246388,0.158629,0.53798,-1
1,-0.402182,-0.211633,0.0,0.0,-0.738752,-0.497202,-0.258563,-0.346181,-0.164315,-0.544335,...,-0.429451,0.342679,0.223577,0.083049,0.445390,0.442750,0.246388,0.158629,0.53798,-1
2,-0.402182,-0.211633,0.0,0.0,-0.738752,-0.497202,-0.258563,-0.346181,-0.164315,-0.544335,...,-0.429451,0.342679,0.223577,0.083049,0.445390,0.442750,0.246388,0.158629,0.53798,-1
3,1.162866,-0.211633,0.0,0.0,0.453797,-0.497202,-0.258563,-0.346181,-0.164315,-0.544335,...,0.527923,-1.807936,0.223577,0.083049,-0.964891,0.442750,0.246388,0.158629,0.53798,1
4,5.858011,-0.211633,0.0,0.0,4.031442,2.476703,-0.258563,2.243146,2.338244,4.132904,...,8.186912,-5.547024,-12.225623,-4.789160,-6.541620,-1.845256,-1.041451,-0.168716,-3.24733,1


In [63]:
from sklearn.linear_model import LogisticRegression, RidgeClassifier, Lasso
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()
#Lasso(alpha=0.01)
model.fit(train_df.iloc[:,:-1], train_df.iloc[:,-1].values.reshape(-1))

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [64]:
tr_acc = model.score(train_df.iloc[:,:-1], train_df.iloc[:,-1].values.reshape(-1))
print('{0:.2f}% training accuracy'.format(tr_acc*100))
val_acc = model.score(val_df.iloc[:,:-1], val_df.iloc[:,-1].values.reshape(-1))
print('{0:.2f}% validation accuracy'.format(val_acc*100))

98.65% training accuracy
96.45% validation accuracy


In [66]:
#get coefficients - logistic
#[(feats_list[i],model.coef_[0][i]) for i in list(range(len(feats_list)))]

#coefficients - lasso
#model.coef_


In [67]:
true_label = val_df.iloc[:,-1]
prediction = model.predict(val_df.iloc[:,:-1])
matrix = confusion_matrix(true_label,prediction)
print('confusion matrix: \n{}\n'.format(matrix))
tn1, fp1, fn1, tp1 = matrix.ravel()

confusion matrix: 
[[7272  174]
 [ 182 2395]]



In [68]:
#use FNC scorer to generate score report
label_prediction = ["discuss" if i == 1 else "unrelated" for i in prediction]
label_actual = pd.DataFrame(stances_val)['Stance']
score.report_score(label_actual, label_prediction)

-------------------------------------------------------------
|           |   agree   | disagree  |  discuss  | unrelated |
-------------------------------------------------------------
|   agree   |     0     |     0     |    653    |    55     |
-------------------------------------------------------------
| disagree  |     0     |     0     |    156    |    23     |
-------------------------------------------------------------
|  discuss  |     0     |     0     |   1586    |    104    |
-------------------------------------------------------------
| unrelated |     0     |     0     |    174    |   7272    |
-------------------------------------------------------------
Score: 3606.25 out of 4438.5	(81.2492959333108%)


81.2492959333108